# Building an ETL Pipeline

In [110]:
import pandas as pd
import mysql.connector 
from datetime import datetime, date, time, timedelta
import tweepy
import numpy as np 
import pyodbc

### Connecting to Twitter

In [111]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [112]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

In [113]:
def twitter_df():
    api = twitter_setup()
    all_tweets = []
    tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
    df.set_index('Tweets')
    return df
twitter_df()

,Tweets,Date
0,Illegal electricity tokens/vouchers DO NOT reg...,2020-03-04 10:31:00
1,Please report electrical emergencies to 086003...,2020-03-04 08:26:00
2,#POWERALERT 1\nDate: 04 March 2020\n\nNo loads...,2020-03-04 07:05:27
3,"@Faizelpatel7 Apologies, we will resend the link.",2020-03-04 07:02:34
4,Prepaid pricing is based on usage/ consumption...,2020-03-04 06:30:00
5,#EskomGauteng #MediaStatement\n\nDisconnection...,2020-03-03 18:54:53
6,Flooding the home with light. Preparing meals....,2020-03-03 16:00:00
7,#POWERALERT 2\nDate: 03 March 2020\n\nNo loads...,2020-03-03 15:26:13
8,Transformer oil theft has serious consequences...,2020-03-03 14:32:00
9,Meter tampering is when you cause the meter to...,2020-03-03 12:29:00


In [114]:
def extract_municipality_hashtags(df):
    new_df = pd.DataFrame([])
    municipality_dict = { '@CityofCTAlerts' : 'Cape Town',
            '@CityPowerJhb' : 'Johannesburg',
            '@eThekwiniM' : 'eThekwini' ,
            '@EMMInfo' : 'Ekurhuleni',
            '@centlecutility' : 'Mangaung',
            '@NMBmunicipality' : 'Nelson Mandela Bay',
            '@CityTshwane' : 'Tshwane'}
  
    length = len(df['Tweets'])
    mun_list = []
    flag = 0
    for row in df['Tweets']:
        flag = 0
        for key in municipality_dict.keys():
            if key in row:
               mun_list.append(municipality_dict[key])
               flag = 1
            break
        if not flag:
          mun_list.append(np.nan)

    final_hash = []
    for row in df['Tweets']:
     final_hash.append([string for string in row.lower().split() if string[0][0] == '#'])
    final_hash = [np.nan if x == [] else x for x in final_hash]
    
    new_df['Tweets'] = df['Tweets']
    new_df['Date'] = df['Date']
    new_df['municipality'] = mun_list
    new_df['hashtags'] = final_hash
    
    return new_df
extract_municipality_hashtags(twitter_df())

,Tweets,Date,municipality,hashtags
0,Illegal electricity tokens/vouchers DO NOT reg...,2020-03-04 10:31:00,NaN,NaN
1,Please report electrical emergencies to 086003...,2020-03-04 08:26:00,NaN,NaN
2,#POWERALERT 1\nDate: 04 March 2020\n\nNo loads...,2020-03-04 07:05:27,NaN,[#poweralert]
3,"@Faizelpatel7 Apologies, we will resend the link.",2020-03-04 07:02:34,NaN,NaN
4,Prepaid pricing is based on usage/ consumption...,2020-03-04 06:30:00,NaN,NaN
5,#EskomGauteng #MediaStatement\n\nDisconnection...,2020-03-03 18:54:53,NaN,"[#eskomgauteng, #mediastatement]"
6,Flooding the home with light. Preparing meals....,2020-03-03 16:00:00,NaN,[#useelectricitysmartly]
7,#POWERALERT 2\nDate: 03 March 2020\n\nNo loads...,2020-03-03 15:26:13,NaN,[#poweralert]
8,Transformer oil theft has serious consequences...,2020-03-03 14:32:00,NaN,NaN
9,Meter tampering is when you cause the meter to...,2020-03-03 12:29:00,NaN,NaN


### Connecting Data Base

In [115]:
def pyodbc_twitter(connection, df, twitter_table):

  ### Code Here
    conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                     user='sa')
    cur = conn.cursor()

# Creating a table
sql ='''CREATE TABLE twitter_table(
   Date DATE ,
   Tweets TEXT,
)'''

# Insert records into table
"""
INSERT INTO twitter_table
"""
cursor.execute(sql)

#Closing the connection
conn.close()
return None

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC SQL Server Driver][SQL Server]There is already an object named 'twitter_table' in the database. (2714) (SQLExecDirectW)")

In [ ]:
cur.execute(
      """
      CREATE TABLE twitter_table (
      Date ' YYYY-MM-DD HH:MI:SS',Tweets;
      """
)

In [ ]:
cur.execute("""SELECT * FROM twitter_table;""")
row1 = cur.fetchall()
print(row1)

In [ ]:
#conn.commit()

In [ ]:
#cur.execute("""select * from Station_Type""")

In [ ]:
#df = pd.read_sql_query('select*from twitter_table',conn)
#df.head()

In [ ]:
# MS SQL execute script
cursor.execute("""select * from Actors a
join Casts c
on c.actor_id = a.actor_id
""")
row = cursor.fetchall()

In [ ]:
#df = pd.read_sql_query('select*from dbo.Actors',conn)

In [ ]:
#df.head()

In [ ]:
a = np.asarray(row)
print(a)

In [ ]:
df = pd.DataFrame(a,columns = ['actor_id','actor_name','gender','movie_id','actor_id','characters'])
df.head()

### Insert table to data base

In [ ]:
conn = pyodbc.connect(driver='',
                     host='',
                     trusted_connection='',
                     user='',
                     autocommit=True)
cursor = conn.cursor()

### SOURCE CODE TO BE ALTERED

In [ ]:
def MySQLConnect(self,query):
        """
        Connects to database and extracts
        raw tweets and any other columns we
        need
        Parameters:
        ----------------
        arg1: string: SQL query
        Returns: pandas dataframe
        ----------------
        """

        try:
            con = mysql.connector.connect(host = self.host, database = self.database, \
                user = self.user, password = self.password, charset = 'utf8')

            if con.is_connected():
                print("Successfully connected to database")

                cursor = con.cursor()
                query = query
                cursor.execute(query)

                data = cursor.fetchall()
                # store in dataframe
                df = pd.DataFrame(data,columns = ['date', 'tweet'])
                #print(df.head())



        except Error as e:
            print(e)

        cursor.close()
        con.close()

        
        return df #None